In [10]:
# pip3 install logger https://github.com/UBTEDU/Yan_ADK/archive/latest.tar.gz
import openadk
from openadk.rest import ApiException
from openadk.models.motions_parameter import MotionsParameter
from openadk.models.motions_operation import MotionsOperation
from openadk.api.motions_api import MotionsApi
from loguru import logger
import time

class Robot:
    def __init__(self, node=None):
        self.node = node
        self.configuration = None  # 标识是否连接
        self.is_connected = False

    def connect(self, robot_ip="raspberrypi.local"):
        # 如果未完成，则对robot的调用都弹出提醒
        self.configuration = openadk.Configuration()  # openadk global
        self.configuration.host = f'http://{robot_ip}:9090/v1'  # /ui
        if self.ping_robot() == "online":
            if self.node:
                self.node.pub_notification("Device(Yanshee) Connected!", type="SUCCESS")  # 由一个积木建立连接到时触发
            self.is_connected = True
            return True

    # 装饰器 require connect
    def ping_robot(self):
        if not self.configuration:
            return
        self.devices_api_instance = openadk.DevicesApi(
            openadk.ApiClient(self.configuration))
        try:
            # Get system's battery information
            api_response = self.devices_api_instance.get_devices_battery()
            logger.debug(api_response)
            return "online"
        except ApiException as e:  # global
            error_message = f"Exception when calling DevicesApi->get_devices_battery: {str(e)}"
            logger.error(error_message)
            if node:
                node.pub_notification(error_message, type="ERROR")

    def play(
        self,
        name="wave",  # GetUp, Hug, Bow , HappyBirthday, Forward Backward OneStepForward TurnLeft TurnRight
        operation='start',
        direction='both',
        speed="fast"):  # ['start', 'pause', 'resume', 'stop']

        if not self.configuration:
            return
        motion_api_instance = MotionsApi(openadk.ApiClient(self.configuration))
        timestamp = int(time.time())
        kw = {"name": name, "speed": speed, "repeat": 1}
        if direction:
            kw["direction"] = direction
        motion = MotionsParameter(**kw)
        body = MotionsOperation(motion=motion,
                                operation=operation,
                                timestamp=timestamp)
        try:
            ret = motion_api_instance.put_motions(body)
        except ApiException as e:
            error_message = f"Exception when calling DevicesApi->get_devices_battery: {str(e)}"
            logger.error(error_message)
            if node:
                node.pub_notification(error_message, type="ERROR")
        # assert ret.code == 0
        return ret

    def bow(self):
        if not self.configuration:
            return
        self.play(name="bow", direction="", speed="slow", operation="start")

    def say(self, content="你好"):
        if not self.configuration:
            return
        timestamp = int(time.time())
        api_instance = openadk.VoiceApi(openadk.ApiClient(self.configuration))
        body = openadk.VoiceTTSStr(content)  # VoiceTTSStr |
        api_response = api_instance.put_voice_tts(body)

In [11]:
robot = Robot()

In [12]:
robot.connect("192.168.31.109")

2020-06-02 16:05:23.799 | DEBUG    | __main__:ping_robot:35 - {'code': 0,
 'data': {'charging': 0, 'percent': 93, 'voltage': 4048},
 'msg': 'success'}


True

In [18]:
robot.bow()
robot.say("欢迎来到 codelab")

In [23]:
robot.play(name="OneStepForward")

{'code': 0, 'data': {'total_time': 3350}, 'msg': 'success'}

In [24]:
robot.play(name="Forward")

{'code': 0, 'data': {'total_time': 41430}, 'msg': 'success'}

In [25]:
robot.play(name="Stop")

{'code': 0, 'data': {'total_time': 1790}, 'msg': 'success'}